In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict
import torch
import utils.padding as up
from model.bert import BERT_CRF
import numpy as np

In [2]:
pretrained_model = "google-bert/bert-base-uncased"
data_set = "PassbyGrocer/msra-ner"
length = 32

In [3]:
dataset = load_dataset(data_set)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [4]:
padded_dataset = up.padding_datasets(dataset,tokenizer,32)
padded_dataset

Map:   0%|          | 0/4365 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 46364
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4365
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4365
    })
})

In [5]:
for i in padded_dataset["train"].column_names:
    print(i,np.array(padded_dataset["train"][i]).shape)

tokens (46364, 32)
ner_tags (46364, 32)
input_ids (46364, 32)
token_type_ids (46364, 32)
attention_mask (46364, 32)
labels (46364, 32)


In [6]:
num_labels = len(dataset["train"].features["ner_tags"].feature.names)

In [7]:
pretrained_model = "google-bert/bert-base-uncased"

In [11]:
from transformers import AdamW, DataCollatorForTokenClassification
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification
# Update DataLoader to use the data collator
train_loader = DataLoader(padded_dataset["train"], batch_size=8, shuffle=True, )
val_loader = DataLoader(padded_dataset["validation"], batch_size=8)

In [12]:
model = AutoModelForTokenClassification.from_pretrained(pretrained_model)
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
E:\ProgramData\anaconda3\envs\hf\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
# Training loop
for epoch in range(3):  # Adjust the number of epochs as needed
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = torch.tensor(batch["input_ids"])
        attention_mask = torch.tensor(batch["attention_mask"])
        labels = torch.tensor(batch["labels"])

        optimizer.zero_grad()
        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

C:\Users\sjsun\AppData\Local\Temp\ipykernel_22676\3522875978.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch["input_ids"])
C:\Users\sjsun\AppData\Local\Temp\ipykernel_22676\3522875978.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch["attention_mask"])
C:\Users\sjsun\AppData\Local\Temp\ipykernel_22676\3522875978.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["labels"])
E:\ProgramData\anaconda3\envs\hf\lib\site-packages\TorchCRF\__init

RuntimeError: grad can be implicitly created only for scalar outputs

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(pretrained_model)